! rm -rf metastore_db/

spark.stop()

spark = SparkSession \
    .builder \
    .appName("Prediction of Loan Payment") \
    .master("local[*]") \
    .config("spark.executor.cores","3")\
    .getOrCreate();

In [3]:
import time as t

In [4]:
start_time = t.time()

In [5]:
! rm -rf LoanStats_web.csv

In [6]:
! wget https://storage.googleapis.com/grizzy-lab/LoanStats_web.csv

--2022-04-22 00:35:18--  https://storage.googleapis.com/grizzy-lab/LoanStats_web.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.217.144, 142.250.188.240, 142.251.40.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.217.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1160243241 (1.1G) [text/csv]
Saving to: ‘LoanStats_web.csv’

LoanStats_web.csv   100%[===================>]   1.08G   288MB/s    in 3.9s    

2022-04-22 00:35:22 (284 MB/s) - ‘LoanStats_web.csv’ saved [1160243241/1160243241]



In [7]:
! hdfs dfs -mkdir -p /rawzone/loanstats/

Picked up JAVA_TOOL_OPTIONS: -Dhttps.protocols=TLSv1.2


In [8]:
! hdfs dfs -put LoanStats_web.csv /rawzone/loanstats/

Picked up JAVA_TOOL_OPTIONS: -Dhttps.protocols=TLSv1.2
put: `/rawzone/loanstats/LoanStats_web.csv': File exists


# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [9]:
#! rm -rf LoanStats_web.csv

In [10]:
#! touch LoanStats_web.csv

In [11]:
! ls -l LoanStats_web.csv

-rw-rw-r-- 1 hadoopuser hadoopuser 1160243241 Aug 14  2021 LoanStats_web.csv


In [12]:
! head -3 LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [13]:
! wc -l /root/notebook/data/LoanStats_web.csv

wc: /root/notebook/data/LoanStats_web.csv: Permission denied


In [14]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.load('hdfs://pongsiam-hadoop-master:9000/rawzone/loanstats/*')

In [15]:
raw201617_df = raw_df

In [16]:
raw201617_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

### Numbers of Fields [1]

In [17]:
len(raw201617_df.columns)

144

In [18]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [19]:
len(rawweb_df.columns)

73

In [20]:
rawweb_df.select(['issue_d']).distinct().show()

+--------+
| issue_d|
+--------+
|Oct-2016|
|Feb-2019|
|Mar-2018|
|Sep-2018|
|    null|
|Jan-2016|
|Jul-2016|
|Aug-2016|
|Feb-2017|
|Nov-2018|
|Jul-2018|
|Apr-2018|
|Dec-2018|
|May-2016|
|Jan-2018|
|Aug-2018|
|Jan-2019|
|Apr-2016|
|Sep-2016|
|Oct-2018|
+--------+
only showing top 20 rows



In [21]:
raw_df = rawweb_df

In [22]:
raw_df.count()

1432466

In [23]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [24]:
len(df_colfam5.columns)

22

In [25]:
! pip install pandas

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
import pandas as pd
pd.DataFrame(df_colfam5.describe().take(6), columns=df_colfam5.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432440,15370.388358325654,9646.026272413876,1000,9975
term,1432440,None,None,36 months,60 months
int_rate,1432440,None,None,5.31%,30.99%
installment,1432440,456.6878153569824,281.71344223131814,100.01,999.98
grade,1432440,None,None,A,G
emp_length,1432440,None,None,1 year,n/a
home_ownership,1432440,None,None,ANY,RENT
annual_inc,1432440,81034.5829329675,134183.35696714348,0,99999.84
verification_status,1432440,None,None,Not Verified,Verified


df_colfam5.select(['last_credit_pull_d']).distinct().\
orderBy(['last_credit_pull_d'],ascending=1).show()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5.last_credit_pull_d == '').count()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5['last_credit_pull_d'].isNull()).count()

In [27]:
df_colfam5.groupby('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|          null|    26|
|           OWN|170888|
|          RENT|556961|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



# 2. Data Preparation

Divide this process into 2 parts. The first is a business oriented preparation that turn many business rules to be programming's logics. Its result benefits many tasks related to the Business Intelligence and other descriptive analytics. The second one is a data science oriented preparation that turn many requirements of data science to be programming's logics. Its result benefits many tasks related to the Predictive analytics.

Asumption: Business oriented preparation: No missing values, ONLY month, Correct data types.

## Data Cleansing: Remove missing values

In [28]:
df_no_missing = df_colfam5.dropna(how='any')

## Check for a schema

In [29]:
df_no_missing.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [30]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [31]:
df_no_missing_cached = df_no_missing_fitmem.cache()

In [32]:
df_no_missing_cached.registerTempTable("df")

## Number of data rows

In [33]:
df_no_missing_cached.count()

1429645

## Data Transformation: Remove Sign of Percent and Extract Month.

In [34]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [35]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [36]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [37]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [38]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [39]:
extractterm = udf(lambda x: x.replace('months',''),StringType())

In [40]:
#extractemp_length = udf(lambda x: x.replace('year',''),StringType())

In [41]:
from pyspark.sql.functions import col

In [42]:
dfWithCrunch = df_no_missing_cached.\
withColumn('revol_util',removepercentsign(col('revol_util')).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(col('int_rate')).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(col('earliest_cr_line')).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(col('last_credit_pull_d')).cast(StringType())).\
withColumn('dti',col('dti').cast(DoubleType())).\
withColumn('loan_amnt',col('loan_amnt').cast(DoubleType())).\
withColumn('revol_bal',col('revol_bal').cast(DoubleType())).\
withColumn('term',extractterm(col('term')).cast(DoubleType())).\
withColumn('installment',col('installment').cast(DoubleType())).\
withColumn('open_acc',col('open_acc').cast(DoubleType())).\
withColumn('total_acc',col('total_acc').cast(DoubleType())).\
withColumn('pub_rec',col('pub_rec').cast(DoubleType())).\
withColumn('annual_inc',col('annual_inc').cast(DoubleType()))

In [43]:
#dfWithCrunch.select(['last_credit_pull_d']).printSchema()

In [44]:
#dfWithCrunch.groupby('emp_length').count().show()

In [45]:
#dfWithCrunch.count()

In [46]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade','term']).show()

+---------+--------+-----+--------------+-----+----+
|loan_amnt|int_rate|  dti|home_ownership|grade|term|
+---------+--------+-----+--------------+-----+----+
|   5000.0|    8.24|21.22|          RENT|    B|36.0|
|   3650.0|   13.49|31.17|          RENT|    C|36.0|
|   7700.0|   11.49|15.99|      MORTGAGE|    B|36.0|
|  26200.0|   11.49|24.26|          RENT|    B|36.0|
|   3000.0|   18.99|10.75|      MORTGAGE|    D|36.0|
|  32150.0|   22.74|23.54|      MORTGAGE|    E|60.0|
|  35000.0|   11.39|14.69|      MORTGAGE|    B|36.0|
|  14000.0|    8.24|21.67|          RENT|    B|36.0|
|  15000.0|    7.49| 8.87|      MORTGAGE|    A|36.0|
|  15000.0|   23.99|26.42|      MORTGAGE|    E|60.0|
|  18000.0|   17.99|14.14|          RENT|    D|60.0|
|  18250.0|   16.99|32.42|          RENT|    D|36.0|
|   4800.0|    7.49|14.98|      MORTGAGE|    A|36.0|
|  15000.0|   13.99|15.64|      MORTGAGE|    C|36.0|
|   3000.0|   12.74| 31.8|      MORTGAGE|    C|36.0|
|   5000.0|   11.49|21.91|          RENT|    B

In [47]:
#dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [48]:
dfWithCrunch.select(['loan_amnt','annual_inc']).describe().show()

+-------+------------------+------------------+
|summary|         loan_amnt|        annual_inc|
+-------+------------------+------------------+
|  count|           1429645|           1429645|
|   mean|15366.699302973815| 81111.66344812175|
| stddev| 9641.851365996217|134222.54004920382|
|    min|            1000.0|               0.0|
|    max|           40000.0|             6.1E7|
+-------+------------------+------------------+



In [49]:
rawhive_df = dfWithCrunch.repartition(60).cache()
#rawhive_df.registerTempTable("crunched_data")

In [50]:
len(rawhive_df.columns)

22

In [51]:
rawhive_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [52]:
#rawhive_df.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [53]:
rawhive_df.count()

1429645

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [54]:
from pyspark.sql.functions import *

max_annual_inc = rawhive_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = rawhive_df.select(min('annual_inc')).collect()[0][0]

#sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [55]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [56]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [57]:
max_loan_amnt = rawhive_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = rawhive_df.select(min('loan_amnt')).collect()[0][0]

#sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [58]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [59]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [60]:
normalized_df = rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(col('loan_amnt'))).\
withColumn('annual_inc',normalized_annual_inc(col('annual_inc')))

In [61]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



### Number of data rows that are only "Fully Paid" and "Charged Off"

In [62]:
from pyspark.sql.functions import col

In [63]:
normalized_df.select(col('loan_status')).groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|582923|
|           Default|  1416|
|   In Grace Period|  6125|
|       Charged Off|152817|
|Late (31-120 days)| 19405|
|           Current|661982|
| Late (16-30 days)|  4977|
+------------------+------+



In [64]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|loan_amnt|term|int_rate|installment|grade|          annual_inc|revol_bal|loan_status|
+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|      0.0|36.0|    5.31|      30.12|    A|4.098360655737705E-4|   3215.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|5.081967213114754E-4|  25993.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|5.901639344262295E-4|   3311.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.065573770491804E-4|   1605.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1329.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1016.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|7.180327868852459E-4|    804.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|9.344262295081968E-4|    858.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|

In [65]:
normalized_df.select(['loan_amnt','annual_inc']).describe().show()

+-------+-------------------+--------------------+
|summary|          loan_amnt|          annual_inc|
+-------+-------------------+--------------------+
|  count|            1429645|             1429645|
|   mean|0.36837690520448657|0.001329699400788...|
| stddev| 0.2472269581024671|0.002200369509003...|
|    min|                0.0|                 0.0|
|    max|                1.0|                 1.0|
+-------+-------------------+--------------------+



In [66]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [67]:
data = normalized_filtered_df.repartition(60).cache()

### Drop Null

In [68]:
data_no_missing_df = data.dropna(how='any')

In [69]:
data_no_missing_df.count()

735740

In [70]:
df_no_missing_cached.select(['loan_status']).distinct().show(100)

+------------------+
|       loan_status|
+------------------+
|        Fully Paid|
|           Default|
|   In Grace Period|
|       Charged Off|
|Late (31-120 days)|
|           Current|
| Late (16-30 days)|
+------------------+



In [71]:
data_no_missing_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



# 3. Data Modeling

In [72]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder,\
VectorIndexer, QuantileDiscretizer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes\
, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [73]:
data_no_missing_df_fully_paid = data_no_missing_df.filter(col('loan_status') == 'Fully Paid').sample(True, 0.3, 42)

In [74]:
data_no_missing_df_charge_off = data_no_missing_df.filter(col('loan_status') == 'Charged Off')

In [75]:
final_data_no_missing_df = data_no_missing_df_fully_paid.union(data_no_missing_df_charge_off)

In [76]:
training , test = final_data_no_missing_df.filter(col('loan_amnt') > 0).randomSplit([0.8,0.2])

In [77]:
print("Getting training and testing set with ", t.time()-start_time, "seconds")

Getting training and testing set with  169.37239241600037 seconds


In [78]:
training.select('loan_amnt').describe().show()

+-------+-------------------+
|summary|          loan_amnt|
+-------+-------------------+
|  count|             261089|
|   mean| 0.3593248043623227|
| stddev|0.23766068497372678|
|    min|6.41025641025641E-4|
|    max|                1.0|
+-------+-------------------+



In [79]:
training.select(['revol_bal']).\
describe().show()

+-------+------------------+
|summary|         revol_bal|
+-------+------------------+
|  count|            261089|
|   mean|15831.228745753364|
| stddev| 21394.24229144329|
|    min|               0.0|
|    max|         1137891.0|
+-------+------------------+



In [80]:
training.select(['loan_amnt','term','int_rate','installment','annual_inc']).\
describe().show()

+-------+-------------------+------------------+-----------------+-----------------+--------------------+
|summary|          loan_amnt|              term|         int_rate|      installment|          annual_inc|
+-------+-------------------+------------------+-----------------+-----------------+--------------------+
|  count|             261089|            261089|           261089|           261089|              261089|
|   mean|0.35932480436232284|42.085832800309475|14.07336774050256|461.6829989390593|0.001277946003326...|
| stddev|0.23766068497372686|10.441410053461487|5.394689719761267| 284.011203064899|0.001308850951058...|
|    min|6.41025641025641E-4|              36.0|             5.31|            31.28|                 0.0|
|    max|                1.0|              60.0|            30.99|          1715.42| 0.15693560655737704|
+-------+-------------------+------------------+-----------------+-----------------+--------------------+



In [81]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

## Calculation for Confusion Matrix.

In [82]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Charged Off')]))
    
    fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Fully Paid')]))

    tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Fully Paid')]))

    fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Charged Off')]))


    try:
        positivepredictivevalue = tp / (tp+fp)
        negativepredictivevalue = tn / (tn+fn)
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        accuracy = (tp+tn) / (tp+tn+fp+fn)
        return {'PPV': positivepredictivevalue, 'Sensitivity': sensitivity,\
            'NPV':negativepredictivevalue, 'Specificity': specificity, 'Accuracy': accuracy}
    except ZeroDivisionError:
        return 0

    

In [83]:
! pip install pandas

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Jan2020

In [84]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

In [93]:
def get_model(df,categoricalCols,continuousCols,\
              discretedCols,split_range,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    
    labelIndexer = StringIndexer(inputCol=labelCol,\
                             outputCol='indexedLabel',\
                             handleInvalid='keep')

    indexers = [ StringIndexer(handleInvalid='keep',\
                               inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]
    discretizers = [ Bucketizer(inputCol=d, outputCol="{0}_discretized".format(d)\
                 ,splits=split_range)
                 for d in discretedCols ]
    
    
    featureCols = ['features']
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols +\
                                [discretizer.getOutputCol() for discretizer in discretizers], \
                                outputCol='features')
    

    pipeline_feature = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler])
    
    
    
    train_df_features = pipeline_feature.fit(training).transform(training)
    
    layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 5, 3]
    
    
    clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)
    
    
    pipeline = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler] + [clf])
    


    model=pipeline.fit(df)


    return model

In [94]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

In [95]:
training.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|139223|
|Charged Off|121866|
+-----------+------+



In [96]:
training.select('grade').describe().show()

+-------+------+
|summary| grade|
+-------+------+
|  count|261089|
|   mean|  null|
| stddev|  null|
|    min|     A|
|    max|     G|
+-------+------+



In [97]:
catcols = [\
           'emp_length',\
           'home_ownership',\
           'verification_status',\
           #'purpose',\
           'grade',\
           'addr_state'\
          ]

num_cols = [\
            #'dti',\
            'loan_amnt',\
            'int_rate',\
            'installment',\
            'annual_inc',\
            #'revol_bal',\
            #'delinq_2yrs',\
            'pub_rec',\
            #'revol_util'\
            'total_acc'\
           ]

discols = [\
           #'pub_rec',\
           'dti',\
           #'installment'\
          ]



labelCol = 'loan_status'

splits = [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, float("inf")]

In [98]:
from itertools import combinations 
list(combinations(catcols, 2))

[('emp_length', 'home_ownership'),
 ('emp_length', 'verification_status'),
 ('emp_length', 'grade'),
 ('emp_length', 'addr_state'),
 ('home_ownership', 'verification_status'),
 ('home_ownership', 'grade'),
 ('home_ownership', 'addr_state'),
 ('verification_status', 'grade'),
 ('verification_status', 'addr_state'),
 ('grade', 'addr_state')]

In [ ]:
spark_labeling = udf(lambda x: "Charged Off" if x == 1.0 else "Fully Paid")

In [ ]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        ml_model = get_model(training,j\
                             ,num_cols,discols, splits, labelCol)
        lap2 = ml_model.transform(test)\
        .withColumn('prediction_label',spark_labeling(col('prediction')))\
        .toPandas()
        m2 = eval_metrics(lap2)
        print("ใช้ {0} ได้ Model ที่มี {1}".format(j,m2))
        print("Getting MLPC Model with ", t.time()-start_time, "seconds")
        

ใช้ ('emp_length',) ได้ Model ที่มี {'PPV': 0.5467506714525967, 'Sensitivity': 0.7156421460722148, 'NPV': 0.6559195356603323, 'Specificity': 0.4774568815835166, 'Accuracy': 0.5890018925115013}
Getting MLPC Model with  504.73344922065735 seconds
